In [43]:
from cython_bbox import bbox_overlaps as bbox_ious
import numpy as np
import time
from numba import njit

In [54]:
def get_random_boxes():
    num_boxes1 = 100
    boxes1 = np.random.uniform(0, 10, (num_boxes1, 4))
    boxes1[:, 2:] = np.maximum(boxes1[:, :2], boxes1[:, 2:])
    return boxes1

In [69]:
boxes1 = get_random_boxes()
boxes2 = get_random_boxes()

t1 = time.time()

for i in range(1000):
    bbox_ious(
        np.ascontiguousarray(boxes1, dtype=float),
        np.ascontiguousarray(boxes2, dtype=float)
    )

print(time.time() - t1)


0.15335440635681152


In [66]:
@njit(parallel=True)
def nb_box_ious(
        boxes,
        query_boxes):
    N = boxes.shape[0]
    K = query_boxes.shape[0]
    overlaps = np.zeros((N, K), dtype=float)
    for k in range(K):
        box_area = (
            (query_boxes[k, 2] - query_boxes[k, 0] + 1) *
            (query_boxes[k, 3] - query_boxes[k, 1] + 1)
        )
        for n in range(N):
            iw = (
                min(boxes[n, 2], query_boxes[k, 2]) -
                max(boxes[n, 0], query_boxes[k, 0]) + 1
            )
            if iw > 0:
                ih = (
                    min(boxes[n, 3], query_boxes[k, 3]) -
                    max(boxes[n, 1], query_boxes[k, 1]) + 1
                )
                if ih > 0:
                    ua = (boxes[n, 2] - boxes[n, 0] + 1) *(boxes[n, 3] - boxes[n, 1] + 1) +box_area - iw * ih

                    overlaps[n, k] = iw * ih / ua
    return overlaps

In [70]:
boxes1 = get_random_boxes()
boxes2 = get_random_boxes()

t1 = time.time()
for i in range(1000):
    dumb_bbox_overlaps(
        np.ascontiguousarray(boxes1, dtype=float),
        np.ascontiguousarray(boxes2, dtype=float)
    )

print(time.time() - t1)

0.0685880184173584


In [72]:
dumb_bbox_overlaps(
np.ascontiguousarray(np.array([[0,0,1,1], [0.5,0.5,1.5,1.5]]), dtype=float),
np.ascontiguousarray(np.array([[0.75,0.75,1.75,1.75], [0.5,0.55,1.55,1.55]]), dtype=float)
)

array([[0.24271845, 0.36708861],
       [0.62025316, 0.92857143]])